# BAX-422 Project 1
-- Anakin Liu, Jingyu Tang, Ruijie Shan, Ruiyuan Yang

<br/>

## Smart Travel Planner

#### APIs used:
* **OpenWhether** - https://openweathermap.org/api
* **Amadeus** - https://developers.amadeus.com/self-service

#### Problems/Questions Solved:
- **What is the current weather at my destination?**  
  *(Use OpenWeather API to fetch real-time weather data for a given location.)*
- **What are the cheapest flights available from my city?**  
  *(Use Skyscanner API to retrieve flight prices and compare options.)*
- **What are the available hotel options in the desired destination based on customer preferences?**  
  *(Use Amadeus Hotel List API to retrieve hotels that match customer preferences.)*

<br/>

### Libraries

In [1]:
import os
import requests as rq
from datetime import datetime

<br/>

### API Key Extract Function

In [2]:
def ext_api(file_name: str)->dict[str, str]:
    api_dict = {}
    with open(file_name, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            api_dict[key.strip()] = value.strip()
    return api_dict

<br/>

### Functionalities Class

In [56]:
class SmartTravelPlanner:
    def __init__(self, weather_key:str, amadeus_key_pack:tuple[str, str]):
        # Extract API keys
        self.__weather_key = weather_key

        # Base URLs for the APIs
        self.weather_base_url = "https://api.openweathermap.org/data/2.5/weather"
        self.amadeus_auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
        self.amadeus_flight_url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        self.amadeus_hotel_list_url = "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city"
        self.amadeus_hotel_ratings_url = "https://test.api.amadeus.com/v2/e-reputation/hotel-sentiments"
        self.amadeus_hotel_offer_url = "https://test.api.amadeus.com/v3/shopping/hotel-offers"

        # Initialize Amadeus access token
        payload = {
            "grant_type":"client_credentials",
            "client_id":amadeus_key_pack[0],
            "client_secret":amadeus_key_pack[1]
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        response = rq.post(self.amadeus_auth_url, data=payload, headers=headers)
        self.__amadeus_token = response.json().get("access_token")
    


    # Generic method for making API calls and handling errors.
    def make_api_request(self, url:str, params:dict[str, str], headers=None)->dict|None:
        try:
            response = rq.get(url, params=params, headers=headers)
            return response.json()
        except rq.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return None
        
    

    # Method for timestamp formation
    def format_timestamp(self, timestamp:int, is_24h:bool)->str:
        dt_object = datetime.fromtimestamp(timestamp)
        if is_24h:
            return dt_object.strftime("%H:%M")
        else:
            return dt_object.strftime("%I:%M %p")
    


    # Fetch weather data using OpenWeather API
    def get_weather(self, city:str, pref:str='global')->dict[str, str]:
        # Distinguish user preference
        settings = {
            "global":{"metric":"metric", "temp_unit":"C", "dist_unit":"km", "spd_unit":"m/s", "is_24h":True},
            "us":{"metric":"imperial", "temp_unit":"F", "dist_unit":"mile", "spd_unit":"mph", "is_24h":False}
        }
        rule = settings[pref]

        # API call
        params = {"q":city, "appid":self.__weather_key, "units":rule["metric"]}
        json_data = self.make_api_request(self.weather_base_url, params)

        # Convert visibility
        visibility = int(json_data['visibility'] * (0.001 if pref == 'global' else 0.000621371))

        # Convert wind direction
        wind_deg = json_data['wind']['deg']
        wind_directions = ["North", "North-East", "East", "South-East", "South", "South-West", "West", "North-West"]
        wind_dir = wind_directions[round(wind_deg / 45) % 8]

        # Formatting data for presentation
        return {
            "city": city,
            "time": self.format_timestamp(json_data['dt'], rule['is_24h']),
            "temperature": f"{json_data['main']['temp']}°{rule['temp_unit']}",
            "min_temperature": f"{json_data['main']['temp_min']}°{rule['temp_unit']}",
            "max_temperature": f"{json_data['main']['temp_max']}°{rule['temp_unit']}",
            "feels": f"{json_data['main']['feels_like']}°{rule['temp_unit']}",
            "humidity": f"{json_data['main']['humidity']}%",
            "visibility": f"{visibility} {rule['dist_unit']}",
            "weather": json_data['weather'][0]['main'],
            "weather_icon": f"http://openweathermap.org/img/wn/{json_data['weather'][0]['icon']}.png",
            "wind_speed": f"{json_data['wind']['speed']} {rule['spd_unit']}",
            "wind_dir": wind_dir,
            "sunrise": self.format_timestamp(json_data['sys']['sunrise'], rule['is_24h']),
            "sunset": self.format_timestamp(json_data['sys']['sunset'], rule['is_24h'])
        }
    


    # Fetch flight prices from Amadeus API
    def get_flight_offers(
            self, 
            origin:str, 
            destination:str, 
            departure_date:str, 
            adults:int=1, 
            max_results:int=10
        )->list[dict[str, str]]:
        # Error check
        if not self.__amadeus_token:
            return {"error": "Failed to authenticate with Amadeus API"}

        # Parameter setting
        headers = {"Authorization":f"Bearer {self.__amadeus_token}"}
        params = {
            "originLocationCode":origin,
            "destinationLocationCode":destination,
            "departureDate":departure_date,
            "adults":adults,
            "max": max_results
        }

        # API call
        json_data = self.make_api_request(self.amadeus_flight_url, params, headers)['data']

        # Extract important information
        flight_offers = []
        for flight in json_data:
            # Airline and price information
            airline_code = flight["validatingAirlineCodes"][0]
            price = float(flight["price"]['grandTotal'])  # Convert price to float for sorting
            price_unit = flight["price"]['currency']
            
            # Flight itinerary details
            itinerary = flight["itineraries"][0]
            segments = itinerary["segments"]
            # Extract departure and arrival times
            first_segment = segments[0]
            last_segment = segments[-1]
            departure_time = first_segment["departure"]["at"]
            arrival_time = last_segment["arrival"]["at"]
            # Check if the flight is direct or requires transits
            is_direct = len(segments) == 1
            transit_info = 'Direct flight' if is_direct else 'Need to transfer'

            # Append formatted flight information
            flight_offers.append({
                "airline": airline_code,
                "price": price,  # Store as float for sorting
                "price_display": f"{price} {price_unit}",  # Separate display version
                "departure_time": departure_time,
                "arrival_time": arrival_time,
                "transit_info": transit_info,
                "carrier": first_segment["carrierCode"]
            })

        # Sort flights by price (ascending order)
        flight_offers.sort(key=lambda x: x["price"])

        # Clean up
        for flight in flight_offers:
            del flight["price"]

        return flight_offers

    

    # Fetch hotel ratings and pricings from Amadeus hotel API
    def final_hotel_list(
        self, 
        hotel_list:list[dict], 
        adult_count:int,
        room_count:int,
        checkin_date:str,
        checkout_date:str,
        currency:str,
        header:dict[str, str]
    )->list[dict]:
        completed_list = []
        for htl in hotel_list:
            # Pricing API call
            params_pricing = {
                "hotelIds":[htl["hotel_id"]],
                "adults":adult_count, 
                "roomQuantity":room_count,
                "checkinDate":checkin_date,
                "checkOutDate":checkout_date,
                "currency":currency
            }
            json_data_pricing = self.make_api_request(self.amadeus_hotel_offer_url, params_pricing, header)

            # Ignore 404 not found option
            if 'data' not in json_data_pricing.keys():
                continue

            json_data_rating = json_data_rating["data"][0]


            # Ratings API call
            params_rating = {"hotelIds":[json_data_rating["hotel"]["hotelId"]]}
            json_data_rating = self.make_api_request(self.amadeus_hotel_ratings_url, params_rating, header)

            # Ignore 404 not found option
            if 'data' not in json_data_rating.keys():
                continue

            json_data_pricing = json_data_pricing["data"][0]

            # Ignore unavailable option
            if not json_data_pricing["available"]:
                continue


            # Fetch rating info
            htl['rating_count'] = json_data_rating["numberOfRatings"]
            htl['rating'] = json_data_rating["overallRating"]

            # Fetch pricing info
            offer = json_data_pricing["offers"][0]
            htl['room_type'] = offer["room"]["typeEstimated"]["category"]
            htl['bed_count'] = offer["room"]["typeEstimated"]["beds"]
            htl['bed_type'] = offer["room"]["typeEstimated"]["bedType"]
            htl['price'] = f'{offer["price"]["total"]} {offer["price"]["currency"]}'
            htl['payment_type'] = offer["policies"]["paymentType"]

            completed_list.append(htl)
        return completed_list



    # Fetch hotel options from Amadeus hotel API
    def get_hotel_list(
            self, 
            city_code:str, 
            distance:str, 
            dist_unit:str, 
            star:str,
            adults:int,
            rooms:int,
            checkin:str,
            checkout:str,
            currency_code:str)->list[dict]:
        # Error check
        if not self.__amadeus_token:
            return {"error": "Failed to authenticate with Amadeus API"}

        # Parameter setting
        headers = {"Authorization":f"Bearer {self.__amadeus_token}"}
        params = {"cityCode": city_code, "radius":distance, "radiusUnit":dist_unit.upper(), "ratings":star}

        # API call
        json_data = self.make_api_request(self.amadeus_hotel_list_url, params, headers)['data']

        # Filter hotel info
        filtered_hotel_list = []
        for htl in json_data:
            filtered_hotel_list.append({"hotel_id":htl['hotelId'], "hotel_name":htl['name']})

        '''
        # Supplement the database with hotel rating and pricing data
        final_hotel_list = self.final_hotel_list(
            filtered_hotel_list, 
            adults, 
            rooms, 
            checkin, 
            checkout, 
            currency_code, 
            headers
        )
        '''

        return filtered_hotel_list

<br/>

### Function Test - Setup

In [57]:
apis = ext_api('api_keys.txt')

stp = SmartTravelPlanner(apis['openweathermap_api'], (apis['amadeus_api'], apis['amadeus_secret']))

<br/>

### Function Test - Weather Information

In [47]:
city_weather = stp.get_weather("London", pref='us')
print(city_weather)

{'city': 'London', 'time': '01:56 PM', 'temperature': '46.65°F', 'min_temperature': '44.62°F', 'max_temperature': '48.09°F', 'feels': '41.76°F', 'humidity': '86%', 'visibility': '6 mile', 'weather': 'Clouds', 'weather_icon': 'http://openweathermap.org/img/wn/04n.png', 'wind_speed': '10.36 mph', 'wind_dir': 'South-West', 'sunrise': '11:34 PM', 'sunset': '08:54 AM'}


<br/>

### Function Test - Flight Information

In [53]:
flight_info = stp.get_flight_offers('SFO', 'LHR', '2025-03-02')
for flight in flight_info:
    print(flight)

{'airline': 'B6', 'price_display': '357.33 EUR', 'departure_time': '2025-03-02T23:20:00', 'arrival_time': '2025-03-04T05:55:00', 'transit_info': 'Need to transfer', 'carrier': 'B6'}
{'airline': 'WS', 'price_display': '383.34 EUR', 'departure_time': '2025-03-02T13:55:00', 'arrival_time': '2025-03-03T12:50:00', 'transit_info': 'Need to transfer', 'carrier': 'WS'}
{'airline': 'VS', 'price_display': '388.73 EUR', 'departure_time': '2025-03-02T16:40:00', 'arrival_time': '2025-03-03T10:55:00', 'transit_info': 'Direct flight', 'carrier': 'VS'}
{'airline': 'UA', 'price_display': '388.73 EUR', 'departure_time': '2025-03-02T12:50:00', 'arrival_time': '2025-03-03T07:25:00', 'transit_info': 'Direct flight', 'carrier': 'UA'}
{'airline': 'UA', 'price_display': '388.73 EUR', 'departure_time': '2025-03-02T17:25:00', 'arrival_time': '2025-03-03T12:00:00', 'transit_info': 'Direct flight', 'carrier': 'UA'}
{'airline': 'UA', 'price_display': '388.73 EUR', 'departure_time': '2025-03-02T16:27:00', 'arrival_

<br/>

### Function Test - Hotel Information

In [58]:
hotel_list = stp.get_hotel_list('LON', '5', 'mile', '4,5', 1, 1, '2025-03-03', '2025-03-05', 'USD')
for htl in hotel_list:
    print(htl)

{'hotel_id': 'HSXXXAAA', 'hotel_name': 'RT ATHENS'}
{'hotel_id': 'YXLON939', 'hotel_name': 'THE LEVIN HOTEL'}
{'hotel_id': 'YXLONEGT', 'hotel_name': 'EGERTON HOUSE HOTEL'}
{'hotel_id': 'MULONCHL', 'hotel_name': 'MILLENNIUM HOTEL KNIGHTSBRIDGE'}
{'hotel_id': 'JTLON423', 'hotel_name': 'JUMEIRAH CARLTON TOWER'}
{'hotel_id': 'VYLONBER', 'hotel_name': 'THE BERKELEY'}
{'hotel_id': 'PHLONTPH', 'hotel_name': 'THE PELHAM HOTEL'}
{'hotel_id': 'PHLONTGH', 'hotel_name': 'THE GORE HOTEL'}
{'hotel_id': 'WVLON005', 'hotel_name': 'CHEVAL PHOENIX HOUSE'}
{'hotel_id': 'FGLON423', 'hotel_name': 'THE GAINSBOROUGH HOTEL'}
{'hotel_id': 'PHLONCLI', 'hotel_name': 'DRAYCOTT HOTEL PREFERRED BOUTI'}
{'hotel_id': 'WVLONKEX', 'hotel_name': 'THE QUEENS GATE HOTEL'}
{'hotel_id': 'EPLONQUE', 'hotel_name': '54 FIFTY FOUR BOUTIQUE HOTEL'}
{'hotel_id': 'EPLONSLO', 'hotel_name': 'SAN DOMENICO HOUSE'}
{'hotel_id': 'DSLON823', 'hotel_name': 'NUMBER SIXTEEN'}
{'hotel_id': 'WVLONSLO', 'hotel_name': 'SLOANE SQUARE HOTEL'}
{'h

<br/>

### Work & Design Plan

#### **Project Overview**
The **Smart Travel Planner with Real-time Data** is designed to assist travelers by providing real-time information on weather, flights, and hotel information. The project integrates multiple APIs to offer a comfortable user experience for trip planning.

#### **Goals and Objectives**
- Provide travelers with accurate and up-to-date information.
- Simplify the process of comparing flight and hotel options.
- Offer a comfortable experience with intuitive navigation and clear results.

#### **Scope of Work**
- Collect and display real-time weather updates.
- Aggregate and compare flight prices from various sources.
- Present hotel  for informed decision-making.

#### **Project Phases**
1. **Phase 1:** Research and requirement gathering.
2. **Phase 2:** Design and planning.
3. **Phase 3:** Development and implementation.
4. **Phase 4:** Testing and feedback incorporation.
5. **Phase 5:** Deployment and maintenance.

#### **Key Functions**
- Retrieve real-time weather data for the destination.
- Fetch and compare flight prices from the user’s departure city.
- Provide customer-rated hotel options in the destination.

<br/>

### Challenges

#### **API Registration Issues**
Initially, we planned to use the **Skyscanner API** for retrieving flight prices and rental car options. However, during the registration process, we encountered a major issue: the platform required us to create a fully operational website and undergo an approval process. Unfortunately, our application was rejected because the API does not support student use.

#### **Difficulties in Finding Alternative APIs**
After Skyscanner API was no longer an option, we explored various alternatives for car rental recommendations and location-based services. However, most APIs required **subscription fees**, making them impractical for our project. We tested multiple APIs before finally identifying **Amadeus API**, which provided the necessary travel-related information while offering free-tier access.

#### **Lack of Depth in Initial Proposal**
When implementing our original problem set, we felt the project **lacked depth and hierarchy**. The initial version focused on basic travel queries, but after adopting the **Amadeus API**, we were able to refine the problem scope, making it **more structured** by integrating hotel information based on customer preferences and more detailed travel recommendations.

#### **Difficulties in Fetching Hotel Data Efficiently**
When retrieving hotel data from the Amadeus API, we encountered a limitation where we could not fetch all hotels in a specific region in a single request. The API required us to query hotels in batches, otherwise, the program would either fail or return incomplete data due to request limitations. To resolve this, we implemented a batch processing mechanism, ensuring that hotel data was fetched incrementally while adhering to API constraints, preventing errors, and maintaining efficient data retrieval.

#### **Data Integration Across APIs**
- **OpenWeather API** provides real-time weather data using **city names**, while **Amadeus API** uses **airport codes or latitude/longitude**. This discrepancy made direct integration difficult.
- We attempted various solutions, including **external location-matching services** and **manual cross-referencing**.
- Finally, after **multiple attempts** and researching **online resources** on format conversion, we **successfully reconciled the address formats**.
- Additionally, we synchronized data for **future travel dates** and developed a **filtering system** to adjust hotel recommendations based on weather conditions.

<br/>

### Decisions Made

#### **1. Switching from Skyscanner API to Amadeus API**
Due to **Skyscanner API’s restrictions**, we opted for **Amadeus API**, which provides a **comprehensive travel solution**, including **hotel and flight recommendations**. This decision allowed us to maintain our project's original intent while **enhancing its scope**.

#### **2. Refining the Problem Scope**
Initially, our queries focused on individual aspects of travel (**weather, flights, car rentals**). After reevaluating, we adjusted the structure to be **more hierarchical**:
- **Step 1:** Check the **destination weather** *(OpenWeather API)*
- **Step 2:** Find the **option hotels** based on **customer preferences** *(Amadeus API)*
- **Step 3:** Provide **flight options** to the selected destination *(Amadeus API)*

This refined flow makes the **user journey more intuitive** and enhances **decision-making**.

#### **3. Balancing API Costs and Features**
Given that many APIs required **paid subscriptions**, we carefully **selected free-tier options** that provided **sufficient data**. The **Amadeus API** met our needs while offering an **extensive travel dataset** without requiring paid access.
